# Определение наиболее выгодного региона нефтедобычи.

## Описание проекта

Вам предоставлены пробы нефти в трёх регионах. Характеристики для каждой скважины в регионе уже известны. Постройте модель для определения региона, где добыча принесёт наибольшую прибыль.

**Цель исследования:** 
На основе данных геологи разведки выбрать район добычи нефти.


## Загрузка и подготовка данных

### Импортируем библиотеки, которые будем использовать в дальнейшем.

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from scipy import stats as st
import numpy as np

### Загружаем данные

In [2]:
df_0 = pd.read_csv('/datasets/geo_data_0.csv')
df_1 = pd.read_csv('/datasets/geo_data_1.csv')
df_2 = pd.read_csv('/datasets/geo_data_2.csv')

df_list = [df_0, df_1, df_2]

### id надо исключать так как при обучении, он никакой значимости не несет для прогноза. 

In [3]:
for i in range(len(df_list)):
    df_list[i] = df_list[i].drop(['id'], axis=1)

### Смотрим что получилось.


In [4]:
display(df_list[i])

,f0,f1,f2,product
0,-1.146987,0.963328,-0.828965,27.758673
1,0.262778,0.269839,-2.530187,56.069697
2,0.194587,0.289035,-5.586433,62.871910
3,2.236060,-0.553760,0.930038,114.572842
4,-0.515993,1.716266,5.899011,149.600746
...,...,...,...,...
99995,-1.777037,1.125220,6.263374,172.327046
99996,-1.261523,-0.894828,2.524545,138.748846
99997,-1.199934,-2.957637,5.219411,157.080080
99998,-2.419896,2.417221,-5.548444,51.795253


## Обучение и проверка модели

### Готовим функцию для разделения базы на выборки.


In [5]:
def create_objects(df):

    df_train, df_valid = train_test_split(df, test_size=0.25, random_state=12345)

    features_train = df_train.drop(['product'], axis=1)
    target_train = df_train['product']
    features_valid = df_valid.drop(['product'], axis=1)
    target_valid = df_valid['product']
    return (features_train, target_train, features_valid, target_valid)

### Делим базу


In [6]:
features_train_list = []
target_train_list = []
features_valid_list = []
target_valid_list = []

for i in range(len(df_list)):
    features_train, target_train, features_valid, target_valid = create_objects(df_list[i])
    features_train_list.append(features_train)
    target_train_list.append(target_train)
    features_valid_list.append(features_valid)
    target_valid_list.append(target_valid)

### Проверяем результат


In [7]:
for i in range(len(df_list)):
    print ('Регион №', i)
    print ('features_train имеет размерность:', features_train_list[i].shape)
    print ('target_train имеет размерность:', target_train_list[i].shape)
    print ('features_valid имеет размерность:', features_valid_list[i].shape)
    print ('target_valid имеет размерность:', target_valid_list[i].shape)
    print ()

Регион № 0
features_train имеет размерность: (75000, 3)
target_train имеет размерность: (75000,)
features_valid имеет размерность: (25000, 3)
target_valid имеет размерность: (25000,)

Регион № 1
features_train имеет размерность: (75000, 3)
target_train имеет размерность: (75000,)
features_valid имеет размерность: (25000, 3)
target_valid имеет размерность: (25000,)

Регион № 2
features_train имеет размерность: (75000, 3)
target_train имеет размерность: (75000,)
features_valid имеет размерность: (25000, 3)
target_valid имеет размерность: (25000,)



### Учим модель


In [8]:
predictions_valid_list = []
for i in range(len(df_list)):
    model = LinearRegression()
    model.fit(features_train_list[i], target_train_list[i])
    predictions_valid = model.predict(features_valid_list[i])
    predictions_valid_list.append(predictions_valid)
    mse = mean_squared_error(target_valid_list[i], predictions_valid)

    print ('Регион №', i)
    print("RMSE =", mse ** 0.5)
    print("Средний запас предсказанного сырья", predictions_valid.mean())
    print ()

Регион № 0
RMSE = 37.5794217150813
Средний запас предсказанного сырья 92.59256778438035

Регион № 1
RMSE = 0.893099286775617
Средний запас предсказанного сырья 68.728546895446

Регион № 2
RMSE = 40.02970873393434
Средний запас предсказанного сырья 94.96504596800489



### Вывод.
Модель региона №1 более точная, но нефти больше в регионе №2


## Подготовка к расчёту прибыли

### Задаем переменные. Считаем объём сырья для безубыточной разработки.


In [9]:
NUMBER_OF_WELLS = 200
PRICE = 450000
BUDGET = 10000000000
BREAK_EVEN_POINT = (BUDGET/NUMBER_OF_WELLS)/PRICE

print("Объём сырья для безубыточной разработки новой скважины составляет:",  round(BREAK_EVEN_POINT), 'тыс. баррелей')


Объём сырья для безубыточной разработки новой скважины составляет: 111 тыс. баррелей


#### Вывод.
Средний запас сырья полученный на предыдущем шаге меньше точки безубыточности. Все скважины разрабатывать не стоит. Нужно выбрать лучшие.

### Готовим функцию для расчета прибыли


In [10]:
def profit_calculation(predictions_valid, target_valid, count, PRICE):
    predictions_valid = pd.Series(predictions_valid)
    target_valid.reset_index(drop=True, inplace=True)
    predictions_valid_sort = predictions_valid.sort_values(axis=0, ascending=False)
    volume = (target_valid[predictions_valid_sort.index][:count].sum())*PRICE
    return (volume)
    

### Считаем прибыль


In [11]:
for i in range(len(df_list)):
    temp = profit_calculation(predictions_valid_list[i], target_valid_list[i], NUMBER_OF_WELLS, PRICE)
    print ('Регион №', i)
    print("Прибыль для 200 лучших по прогнозу скважин составит:", round(temp/1000000), 'млн. руб.')
    print ()

Регион № 0
Прибыль для 200 лучших по прогнозу скважин составит: 13321 млн. руб.

Регион № 1
Прибыль для 200 лучших по прогнозу скважин составит: 12415 млн. руб.

Регион № 2
Прибыль для 200 лучших по прогнозу скважин составит: 12710 млн. руб.



### Вывод:
По данному сравнению самым прибыльным оказался регион №0


## Расчёт прибыли и рисков 

### Корректируем функцию для расчета прибыли для корректной работы в процедуре Bootstrap.


In [12]:
def profit_calculation_BUDGET (predictions_valid, target_valid, count, PRICE):
    #predictions_valid = pd.Series(predictions_valid)
    #target_valid.reset_index(drop=True, inplace=True)
    predictions_valid_sort = predictions_valid.sort_values(axis=0, ascending=False)
    volume = ((target_valid[predictions_valid_sort.index][:count].sum())*PRICE) - BUDGET
    return (volume)

### Функция для процедуры Bootstrap.


In [13]:
def bootstrap_test (predictions_valid, target_valid, count, PRICE):
    state = np.random.RandomState(12345)
    predictions_valid = pd.Series(predictions_valid)
    target_valid.reset_index(drop=True, inplace=True)
    values = []
    for i in range(1000):
        predictions_valid_subsample = predictions_valid.sample(n = 500, replace=True, random_state=state)
        target_valid_subsample = target_valid[predictions_valid_subsample.index]
        predictions_subsample_sort = predictions_valid_subsample.sort_values(axis=0, ascending=False)
        balans = ((target_valid_subsample[predictions_subsample_sort.index][:count].sum())*PRICE) - BUDGET
        values.append(balans)
    values = pd.Series(values)
    risk_of_losses = ((values < 0).mean())*100
    average_profit = values.mean()
    lower = values.quantile(0.025)
    upper = values.quantile(0.975)
    return (average_profit, lower, upper, risk_of_losses)


### Анализируем в цикле все регионы.


In [14]:
for i in range(len(df_list)):

    average_profit, lower, upper, risk_of_losses = bootstrap_test(predictions_valid_list[i], 
                                                                  target_valid_list[i], NUMBER_OF_WELLS, PRICE)

    print ('Регион №', i)
    print('Средняя прибыль составляет:', round(average_profit/1000000), 'млн. руб.')
    print ('С вероятностью 95% прибыль будет находится в интервале от', round(lower/1000000), 
           'до', round(upper/1000000), 'млн. руб.')
    print('Риск убытков составляет:', risk_of_losses, '%')
    print ()


Регион № 0
Средняя прибыль составляет: 426 млн. руб.
С вероятностью 95% прибыль будет находится в интервале от -102 до 948 млн. руб.
Риск убытков составляет: 6.0 %

Регион № 1
Средняя прибыль составляет: 515 млн. руб.
С вероятностью 95% прибыль будет находится в интервале от 69 до 932 млн. руб.
Риск убытков составляет: 1.0 %

Регион № 2
Средняя прибыль составляет: 435 млн. руб.
С вероятностью 95% прибыль будет находится в интервале от -129 до 970 млн. руб.
Риск убытков составляет: 6.4 %



### Вывод:
Только регион №1 советует требованию что вероятность убытков должна быть меньше 2.5%.


## Общий вывод:
Предлагаем рассмотреть к разработке регион №1 так как он удовлетворяет заданному условию, а именно: вероятность убытков составляет 1.0%, что меньше порогового значения в 2.5%.
